In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

In [6]:
dataset=load_dataset('cnn_dailymail','1.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
print(dataset['train'])

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})


In [28]:
def preprocess_text(text):

# remove stop words and digits

    doc = nlp(text)

    tokens = [token.text.lower() for token in doc if not token.is_stop and token.is_alpha]

    preprocessed_text = " ".join(tokens)

    return preprocessed_text


In [29]:
preprocessed_corpus=[]
corpus=[]

for i in range(100): # only taken 100 examples can take entire dataset which has 287113 records
  text=preprocess_text(dataset['train'][i]['highlights'])
  # text=dataset['train'][i]['highlights']
  doc=nlp(text)
  preprocessed_corpus.extend([chunk.text for chunk in doc.noun_chunks])

print(preprocessed_corpus)

['harry potter star daniel radcliffe', 'monday young actor', 'fritter cash away radcliffe earnings potter films', 'trust fund', 'mentally ill inmates', 'miami', 'forgotten floor judge steven leifman', 'avoidable felonies cnn tours facility patient', 'son president leifman', 'new thought', 'die driver', 'man', 'pickup truck', 'half cut face driver', 'free fall minnesota bridge', 'small polyps', 'procedure worrisome spokesman', 'president', 'powers', 'vice president bush', 'routine colonoscopy camp', 'new nfl chief atlanta falcons owner critical michael vick conduct nfl', 'falcons', 'quarterback', 'vick', 'funding dogfighting operation', 'gamble vick federal court', 'monday', 'future nfl', 'parents beam pride', 'support mom', 'happy know burn center', 'treatment', 'reconstructive surgeries', 'dad', 'youssif', 'aid workers violence', 'cost living drive women prostitution group working raise awareness problem iraq political leaders', 'cnn', 'children', 'children woman', 'medina caracas', '

In [31]:
tfidf = TfidfVectorizer()
context_matrix = tfidf.fit_transform(preprocessed_corpus)


tfidf.get_feature_names_out()

array(['abandoned', 'abortion', 'abuse', ..., 'zealand', 'zoe',
       'zvonareva'], dtype=object)

In [32]:
def cosine_similarity_between_noun_phrases(np1, np2):
    np1_index = preprocessed_corpus.index(np1)
    np2_index = preprocessed_corpus.index(np2)
    cosine_sim = cosine_similarity(context_matrix[np1_index], context_matrix[np2_index])
    return cosine_sim[0][0]

In [33]:
np1 = "harry potter star daniel radcliffe"
np2 = "mentally ill inmates"
similarity = cosine_similarity_between_noun_phrases(np1, np2)
print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

np1 = "harry potter star daniel radcliffe"
np2 =  "fritter cash away radcliffe earnings potter films"
similarity = cosine_similarity_between_noun_phrases(np1, np2)
print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

# np1 = "Harry Potter star Daniel Radcliffe"
# np2 = "Mentally ill inmates"
# similarity = cosine_similarity_between_noun_phrases(np1, np2)
# print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

# np1 = "Harry Potter star Daniel Radcliffe"
# np2 =  "Young actor"
# similarity = cosine_similarity_between_noun_phrases(np1, np2)
# print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

# np1 = "Harry Potter star Daniel Radcliffe"
# np2 =  "Radcliffe's earnings"
# similarity = cosine_similarity_between_noun_phrases(np1, np2)
# print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

# np1 = "Harry Potter star Daniel Radcliffe"
# np2 =  "his cash"
# similarity = cosine_similarity_between_noun_phrases(np1, np2)
# print(f"Cosine similarity between '{np1}' and '{np2}': {similarity}")

Cosine similarity between 'harry potter star daniel radcliffe' and 'mentally ill inmates': 0.0
Cosine similarity between 'harry potter star daniel radcliffe' and 'fritter cash away radcliffe earnings potter films': 0.32140297875584767
